# Script NER/REL/LLM/NLM con spacy

##### 1. import tomo completo y tablas del sparcing

In [1]:
#pdf plumber es wraper de pdfminer
import pdfplumber

pdf_file_path = "informes/Informe-Rettig-tomo1.pdf"

def extract_text_from_pdf(pdf_file_path, start_page, end_page):
    with pdfplumber.open(pdf_file_path) as pdf:
        text = ""
        for page_num in range(start_page - 1, end_page):  # Adjusting to 0-based index
            page = pdf.pages[page_num]
            text += page.extract_text()
    return text


# solo la rm, para dividir entre "RM" y "el resto"
start_page = 139
end_page = 252
pdf_text = extract_text_from_pdf(pdf_file_path, start_page, end_page)


In [3]:
print(pdf_text)

Entre los recintos de detención usados durante 1973 en la Región Metropolitana,
se encuentra el local de calle Londres 38, que antes del 11 de septiembre
perteneció al Partido Socialista, y que más tarde sería usado como centro
clandestino de reclusión por la DINA.
Se dispone de antecedentes que indican que en octubre de 1973, es llevado por
algunas horas al local de calle Londres, un grupo de personas que habían sido
detenidas en la Remodelación San Borja en Santiago y que posteriormente son
trasladadas a la Casa de la Cultura de Barrancas. Del mismo modo, se pudo
establecer que en el mes de diciembre son conducidos al mencionado recinto,
algunos militantes del Partido Comunista a quienes la prensa de la época vinculó
a un supuesto plan subversivo denominado "Leopardo", los que posteriormente
fueron ejecutados.
El recinto denominado Parque Cerrillos, donde anualmente se realiza la Feria
Internacional de Santiago (FISA), también fue empleado transitoriamente como
centro de detención, h

In [4]:
# importamos lista de names y eventos hechas en el sparsing 

import pandas as pd
names = pd.read_csv('data_out/names_rettig.csv')
names_list = names['names'].tolist()
names_list

events = pd.read_csv('data_out/events_rettig.csv')
events_list = events['events'].tolist()



#### 2. spacy first run

In [7]:
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
from collections import Counter

# tokenizer, tagger, parser, y NER en español
nlp = spacy.load("es_core_news_sm")

def generate_summary(text, num_sentences=2):
    # Process the text with SpaCy
    doc = nlp(text)
    
    # Filter out stop words and punctuation
    words = [token.text for token in doc if not token.is_stop and not token.is_punct]
    
    # Compute word frequency
    word_freq = Counter(words)
    
    # Calculate the frequency of each sentence
    sentence_scores = {}
    for sent in doc.sents:
        for word in sent:
            if word.text in word_freq:
                if sent not in sentence_scores:
                    sentence_scores[sent] = word_freq[word.text]
                else:
                    sentence_scores[sent] += word_freq[word.text]
    
    # Select the top sentences based on frequency
    top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    
    # Combine the top sentences into a summary
    summary = ' '.join(str(sent) for sent in top_sentences)
    return summary

# Sample events stored in a list
events = events_list

# Generate summaries for each event
summaries = []
for event in events:
    summary = generate_summary(event)
    summaries.append(summary)

# Bind the original events with the summaries
events_with_summaries = list(zip(events, summaries))

/Users/juanipiquer/anaconda3/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'es_core_news_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
# Export excel

events_with_summaries = pd.DataFrame(events_with_summaries)

events_with_summaries.to_excel('data_out/events_summaries.xlsx')